# DX 704 Week 12 Project
This week's project will revisit the email spam classifier project from week 9 using large language model embeddings instead of custom features.


The full project description and a template notebook are available on GitHub: [Project 12 Materials](https://github.com/bu-cds-dx704/dx704-project-12).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [1]:
!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

--2025-11-24 05:08:37--  https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip [following]
--2025-11-24 05:08:38--  https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15642124 (15M) [application/zip]
Saving to: ‘enron_spam_data.zip’

enron_spam_data.zip 100%[===================>]  14.92M  --.-KB/s    in 0.04s   

2025-11-24 05:08:38 (382 MB/s) - ‘enron_

In [2]:
import pandas as pd

In [4]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14
...,...,...,...,...,...
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30


In [5]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

np.float64(0.5092834262664611)

## Part 2: Download BERT Model

We will use a pre-trained BERT model to extract embedding vectors as described in lesson 2.1 this week.
Here is sample code to download the model from [Hugging Face](https://huggingface.co/) and extract one vector.
This model is small enough that you can run it with CPU only, but GPUs will be faster if available.

In [6]:
# You may need to install torch and transformers.
# Google Colab will have these installed already.
#
# pip install transformers torch --upgrade

import torch
from transformers import AutoTokenizer, AutoModel

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

To download the pre-trained model from Hugging Face, you will need to sign up for a free account with them at https://huggingface.co/join.
Afterwards, get an API token and if you are using Google Colab, save it as a secret named HF_TOKEN.

In [8]:
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)
bert_model.to(device)
bert_model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [9]:
@torch.no_grad()
def embed_text(text):
    batch = [text]
    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
    outputs = bert_model(**inputs)
    # CLS token embedding is the first token's hidden state
    cls_emb = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, 768]
    return cls_emb.cpu()

In [10]:
v = embed_text("Hi, will you buy my spam?")
v.shape

torch.Size([1, 768])

## Part 3: Create Embedding Vectors

Use BERT to create embeddings for each email in the Enron data set.
You will have to decide how to combine the different columns of the original data set to produce one embedding vector.


Hint: BERT can be run without a GPU, but will be much slower.
Using Google Colab with only a CPU, it runs around 1 embedding per second.
Using Google Colab with the T4 GPU option, it runs around 60 embeddings per second.
Caching is also encouraged to avoid unnecessary reruns.

In [11]:
# YOUR CHANGES HERE

import json
import numpy as np
import pandas as pd
import torch

# Combine Subject and Message into a single text field
def make_email_text(row):
    subject = str(row["Subject"]) if pd.notna(row["Subject"]) else ""
    body = str(row["Message"]) if pd.notna(row["Message"]) else ""
    # You can tweak this template if you want
    return f"Subject: {subject}\n\nBody: {body}"

texts = enron_spam_data.apply(make_email_text, axis=1).tolist()

len(texts), texts[0]


(33716, 'Subject: christmas tree farm pictures\n\nBody: ')

In [12]:
# Batched embedding with BERT CLS vector

@torch.no_grad()
def embed_batch(text_batch):
    inputs = tokenizer(
        text_batch,
        padding=True,
        truncation=True,
        max_length=256,          # adjust if you want longer context
        return_tensors="pt"
    ).to(device)
    outputs = bert_model(**inputs)
    cls_emb = outputs.last_hidden_state[:, 0, :]  # [batch_size, 768]
    return cls_emb.cpu().numpy()

batch_size = 64   # shrink if you run out of memory
all_embeddings = []

for start in range(0, len(texts), batch_size):
    end = start + batch_size
    batch_texts = texts[start:end]
    batch_embs = embed_batch(batch_texts)
    all_embeddings.append(batch_embs)

all_embeddings = np.vstack(all_embeddings)   # shape: [n_messages, 768]
all_embeddings.shape


(33716, 768)

In [13]:
# Build output DataFrame and save as embeddings.tsv.gz

embedding_json = [json.dumps(vec.tolist()) for vec in all_embeddings]

embeddings_df = pd.DataFrame(
    {
        "Message ID": enron_spam_data["Message ID"].values,
        "embedding_vector_json": embedding_json,
    }
)

embeddings_df.head()


,Message ID,embedding_vector_json
0,0,"[-0.4413585662841797, -0.3294885456562042, -0...."
1,1,"[-0.31733956933021545, -0.02731911465525627, 0..."
2,2,"[-0.44399401545524597, -0.2336771935224533, -0..."
3,3,"[-0.39856159687042236, -0.11958741396665573, -..."
4,4,"[-0.7561951875686646, 0.04038950428366661, 0.1..."


Save your embeddings in a file "embeddings.tsv.gz" with two columns, Message ID and embedding_vector_json, where embedding_vector_json is a JSON-encoded list.
Make sure that embedding_vector_json is a 1 dimensional list, not 2 dimensional.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [14]:
# YOUR CHANGES HERE

embeddings_df.to_csv(
    "embeddings.tsv.gz",
    sep="\t",
    index=False
)


Submit "embeddings.tsv.gz" in Gradescope.

In [16]:

# After you have all_embeddings as a NumPy array of shape [n_messages, 768]
print(all_embeddings.shape)

# 1) Round to 3 decimal places
all_embeddings = np.round(all_embeddings, 3)

# 2) Build JSON strings from 1D vectors
embedding_json = [json.dumps(vec.tolist()) for vec in all_embeddings]

embeddings_df = pd.DataFrame(
    {
        "Message ID": enron_spam_data["Message ID"].values,
        "embedding_vector_json": embedding_json,
    }
)




(33716, 768)


In [17]:
embeddings_df.head()

,Message ID,embedding_vector_json
0,0,"[-0.4410000145435333, -0.32899999618530273, -0..."
1,1,"[-0.31700000166893005, -0.027000000700354576, ..."
2,2,"[-0.4440000057220459, -0.23399999737739563, -0..."
3,3,"[-0.39899998903274536, -0.11999999731779099, -..."
4,4,"[-0.7559999823570251, 0.03999999910593033, 0.1..."


In [18]:
# 3) Save to gzipped TSV
embeddings_df.to_csv(
    "embeddings.tsv.gz",
    sep="\t",
    index=False
)

## Part 4: Train a Linear Regression

Train an ordinary least squares regression for spam/ham status where spam is treated as target value 1 and ham is treated as target value 0 with your embeddings above as the only input variables.


In [19]:
# YOUR CHANGES HERE
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load embeddings and labels
df = embeddings_df.copy()

# Convert JSON list into numpy vectors
X = np.vstack(df["embedding_vector_json"].apply(json.loads).values)

# Spam = 1, Ham = 0
y = (enron_spam_data["Spam/Ham"] == "spam").astype(int).values

print(X.shape, y.shape)  # should be (33716, 768)


(33716, 768) (33716,)


In [20]:
model = LinearRegression()
model.fit(X, y)

print("Model fitted.")
print("Coefficient shape:", model.coef_.shape)
print("Intercept:", model.intercept_)

Model fitted.
Coefficient shape: (768,)
Intercept: 1.193674427729459


Save the coefficients of your linear model in a file "coefficients.tsv" with columns dim and coefficient where dim specifies the offset in the embedding vector (0-767).
Don't worry about the bias term (but your model should still have it).

In [21]:
# YOUR CHANGES HERE

# Create dim numbers 0 to 767
dims = np.arange(model.coef_.shape[0])

coeff_df = pd.DataFrame({
    "dim": dims,
    "coefficient": model.coef_
})

coeff_df.to_csv("coefficients.tsv", sep="\t", index=False)


Submit "coefficients.tsv" in Gradescope.

## Part 5: Search for Relevant Documents

The file "queries.tsv" specifies 10 queries.
For each of the queries, encode them as a vector, and find the message that is closest using $L_2$.

In [23]:
# YOUR CHANGES HERE

import json
import numpy as np
import pandas as pd

# Load queries
queries = pd.read_csv("queries.tsv", sep="\t")
print(queries.head())
print("Columns:", list(queries.columns))


   query_id                                              query
0         1                            accounting arrangements
1         2                       sales higher than production
2         3  asked lawyer to write letter about unexpected ...
3         4                                   engineering exam
4         5                                 discounted tickets
Columns: ['query_id', 'query']


Save your results in a file "query-matches.tsv" with columns query_id, query_vector_json, and Message ID.

In [24]:
# YOUR CHANGES HERE

# Pick query text column
if "query" in queries.columns:
    query_text_col = "query"
elif "text" in queries.columns:
    query_text_col = "text"
else:
    # fall back to the second column if needed
    query_text_col = queries.columns[1]

query_texts = queries[query_text_col].astype(str).tolist()

# Embed all queries in one batch
query_embs = embed_batch(query_texts)    # uses your embed_batch from before
query_embs = np.round(query_embs, 3)     # round to 3 decimals

# Helper so JSON really keeps 3 decimals
def fmt_vector(v):
    return [float(f"{x:.3f}") for x in v]

query_vector_json = [json.dumps(fmt_vector(vec)) for vec in query_embs]


In [25]:
# Rebuild document embedding matrix from embeddings_df
doc_embs = np.vstack(embeddings_df["embedding_vector_json"].apply(json.loads).values)
doc_ids = embeddings_df["Message ID"].values

doc_embs.shape, len(doc_ids)


((33716, 768), 33716)

In [26]:
matches = []

# Use existing query_id column if present, otherwise 0..n-1
if "query_id" in queries.columns:
    query_ids = queries["query_id"].values
else:
    query_ids = np.arange(len(queries))

for i, q_vec in enumerate(query_embs):
    # L2 distance to all documents
    dists = np.linalg.norm(doc_embs - q_vec, axis=1)
    best_idx = np.argmin(dists)
    best_message_id = doc_ids[best_idx]

    matches.append({
        "query_id": int(query_ids[i]),
        "query_vector_json": query_vector_json[i],
        "Message ID": int(best_message_id),
    })

len(matches)


10

In [27]:
matches_df = pd.DataFrame(matches)
matches_df.head()


,query_id,query_vector_json,Message ID
0,1,"[-0.067, 0.486, -0.697, 0.012, 0.353, 0.001, -...",14635
1,2,"[-0.865, -0.246, 0.211, -0.16, -0.095, 0.163, ...",2758
2,3,"[-0.298, -0.203, -0.198, 0.104, -0.608, -0.122...",5057
3,4,"[-0.215, 0.199, -0.022, 0.005, 0.379, -0.211, ...",13222
4,5,"[-0.196, -0.049, -0.286, 0.084, 0.672, -0.281,...",31770


In [28]:
matches_df.to_csv("query-matches.tsv", sep="\t", index=False)


Submit "query-matches.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.